# AI CA4
<br>
sajjad pakdaman savoji
<br>
810195517

# <font color="blue"><b> Question 1 </b></font>  : Decision Tree

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as split
from sklearn import tree
from sklearn.metrics import accuracy_score as acc
from scipy.stats import mode
import operator
import graphviz

## load data , split data 

In [8]:
data = pd.read_csv('data.csv')
train , test = split(data , test_size = 53/303 , random_state = 12347 , shuffle = True)

In [9]:
train_label = train.target
train_feature = train.drop(['target'] , axis = 1)

In [10]:
test_label = test.target
test_feature = test.drop(['target'] , axis = 1)

## decision Tree model
Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.<br>
https://scikit-learn.org/stable/modules/tree.html

In [11]:
dt = tree.DecisionTreeClassifier()
dt.fit(train_feature , train_label);

## DT acc on test data 

In [12]:
pred = dt.predict(test_feature)
print(acc(pred , test_label))

0.7924528301886793


## DT acc on train data 

In [28]:
pred = dt.predict(train_feature)
print(acc(pred , train_label))

1.0


## total performance of decision tree 

In [13]:
res = []
for i in range(1000):
    data = pd.read_csv('data.csv')
    train , test = split(data , test_size = 53/303 , random_state = 12347 , shuffle = True)

    train_label = train.target
    train_feature = train.drop(['target'] , axis = 1)

    test_label = test.target
    test_feature = test.drop(['target'] , axis = 1)

    # decision Tree model

    dt = tree.DecisionTreeClassifier()
    dt.fit(train_feature , train_label);

    ## acc on test data 

    pred = dt.predict(test_feature)
    res.append(acc(pred , test_label))
    
res = np.array(res)
print(res.mean())
print(res.std())
    

0.7529245283018867
0.03127739098613014


# <font color="blue"><b> Question 2 </b></font>  : Steps to impliment Random Forest

## <font color = 'green'>2.1 </font> make 5 samples of data

In [90]:
N_CLF = 5
SAMPLE_SZ  = 150

## <font color="green">2.2</font> bagging 

In [16]:
class Bagging():
    def __init__(self, model = tree.DecisionTreeClassifier , n_clf = 5):
        self.models = [model() for _ in range(n_clf)]
        self.N_CLF = n_clf
        self.samples_features = []
        self.samples_labels = []
        
    def fit(self , features , labels , sample_sz = 150):
        for i in range(self.N_CLF):
            si = features.sample(150 , replace = True)
            li = labels.loc[si.index]

            self.samples_features.append(si)
            self.samples_labels.append(li)
            
        for i in range(self.N_CLF):
            self.models[i].fit(self.samples_features[i] , self.samples_labels[i])
            
    def predict(self , features):
        pred = []
        for model in self.models:
            pred.append(np.array(model.predict(features)))
        
        pred = np.array(pred)
        return mode(pred , axis = 0)[0][0]
        
        

In [17]:
ensemble = Bagging()
ensemble.fit(train_feature , train_label)
pred = ensemble.predict(test_feature)
print(acc(pred , test_label))

0.8113207547169812


### Bagging preformance  

In [18]:
accuracy = []
for i in range(1000):
    ensemble = Bagging()
    ensemble.fit(train_feature , train_label)
    pred = ensemble.predict(test_feature)
    accuracy.append(acc(pred , test_label))
    
accuracy = np.array(accuracy)
print(accuracy.mean())
print(accuracy.std())

0.8100754716981132
0.04044802491876607


## <font color = 'green'>2.3</font> backward elimination 

In [19]:
res = {}
model = tree.DecisionTreeClassifier
for feature in train_feature.columns :
    res_f = []
    for i in range(500):
        train_feature_i = train_feature.drop(feature , axis = 1)
        test_feature_i = test_feature.drop(feature , axis = 1)

        mi = model()
        mi.fit(train_feature_i , train_label)
        predi = mi.predict(test_feature_i)
        res_f.append(acc(predi , test_label))
    res_f = np.array(res_f)
    res[feature] = res_f.mean()

In [20]:
print(res)

{'age': 0.8008679245283019, 'sex': 0.7969056603773585, 'cp': 0.8242264150943396, 'trestbps': 0.7983773584905661, 'chol': 0.725433962264151, 'fbs': 0.8073584905660377, 'restecg': 0.7425660377358491, 'thalach': 0.7447169811320755, 'exang': 0.732943396226415, 'oldpeak': 0.7233584905660377, 'slope': 0.7481509433962263, 'ca': 0.7970188679245285, 'thal': 0.7310188679245283}


In [21]:
max(res.items(), key=operator.itemgetter(1))[0]

'cp'

# <font color = "green">2.4</font> random selection of features 

In [25]:
columns = np.array(train_feature.columns)
selected = np.random.choice(columns , size = 5 , replace = False)
selected

array(['oldpeak', 'exang', 'slope', 'age', 'sex'], dtype=object)

In [26]:
train_feature_selected = train_feature[selected]
test_feature_selected = test_feature[selected]

In [27]:
model = tree.DecisionTreeClassifier
mi = model()
mi.fit(train_feature_selected , train_label)
pred = mi.predict(test_feature_selected)
print(acc(pred , test_label))

0.7547169811320755


### random selection performance 

In [28]:
res = []
for i in range(3000):
    #select some random features
    columns = np.array(train_feature.columns)
    selected = np.random.choice(columns , size = 5 , replace = False)
    
    #only take those features that was selected
    train_feature_selected = train_feature[selected]
    test_feature_selected = test_feature[selected]
    
    #choose some model and train it
    model = tree.DecisionTreeClassifier
    mi = model()
    mi.fit(train_feature_selected , train_label)
    
    #predict using model and record the acc
    pred = mi.predict(test_feature_selected)
    res.append(acc(pred , test_label))
    
res = np.array(res)
print(res.mean())
print(res.std())

0.7125534591194969
0.07143346943939437


# <font color = "green">2.5 </font> Random forest implimentation
https://towardsdatascience.com/understanding-random-forest-58381e0602d2

In [29]:
class RandomForest():
    def __init__(self , n , model = tree.DecisionTreeClassifier):
        self.n = n
        self.model = model
        self.models = [self.model() for _ in range(self.n)]
        self.samples_features = []
        self.samples_labels = []
        
    def fit(self , features , labels):
        # this part is for randomness in features
        all_features = features.columns
        max_feature = len(all_features)
        num_features = [np.random.randint(low = 1 , high = max_feature + 1) 
                        for _ in range (self.n)]
        self.selected_features = [np.random.choice(all_features , size = num_features[i] 
                                              ,replace = False) for i in range (self.n)]
        
        # this part is for bagging
        for i in range(self.n):
            si = features.sample(int(0.3*len(features)) , replace = True)
            li = labels.loc[si.index]

            # chooze only selected feaures
            si = si[self.selected_features[i]]
            
            
            self.samples_features.append(si)
            self.samples_labels.append(li)
            
        # then fit the models
        for i in range(self.n):
            self.models[i].fit(self.samples_features[i] , self.samples_labels[i])
            
    def predict(self , features):
        pred = []
        for i , model in enumerate(self.models):
            features_i = features[self.selected_features[i]]
            pred.append(np.array(model.predict(features_i)))
        
        pred = np.array(pred)
        return mode(pred , axis = 0)[0][0]

In [34]:
forest = RandomForest(10) 
forest.fit(train_feature , train_label)

In [35]:
pred = forest.predict(test_feature)
print(acc(pred , test_label))

0.8490566037735849


###  random forest performance  

In [36]:
res = []
for i in range(100):
    forest = RandomForest(50) 
    forest.fit(train_feature , train_label)
    pred = forest.predict(test_feature)
    res.append(acc(pred , test_label))
    
res = np.array(res)
print(res.mean())
print(res.std())

0.8549056603773585
0.02807392468856127


# <font color="blue"><b> Report </b></font>

## <font color = "green"> <b>Question 1: </b></font> Bootstraping

When training, each tree in a random forest learns from a random sample of the data points. The samples are drawn with replacement, known as bootstrapping, which means that some samples will be used multiple times in a single tree.
<br><br>

bootstraping increases standard deviation(and variance) of accuracy of model because additional randomness is forced to model resulting less deterministic behaviour
<br><br>

boostraping adds bias error thus improoving variance error



## <font color = "green"> <b>Question 2: </b></font> Overfiting and Bagging
Over-fitting is the phenomenon in which the learning system tightly fits the given training data so much that it would be inaccurate in predicting the outcomes of the untrained data. In decision trees, over-fitting occurs when the tree is designed so as to perfectly fit all samples in the training data set.<br><br>
In decision tree , if depth of tree is not limited and entropy gain is used as a discrimianant refrence for feature selection , tree will have a 100% accuracy on train data , in contrast performance of model would not be as perfect.
By implimenting Baggin additional random ness is forced to model resulting to better generalization and worse accuracy on train data itself.This additive randomness is the classic bias-variance trade off struggle

## <font color = "green"> <b>Question 3: </b></font> Random Forest and Bagging

The fundamental difference is that in Random forests, only a subset of features are selected at random out of the total and the best split feature from the subset is used to split each node in a tree, unlike in bagging where all features are considered for splitting a node.<br>
In a more widesense comparison Random Forest uses Bagging to overcome overfiting in decision tree

Random forest, like its name implies, consists of a large number of individual decision trees that operate as an ensemble. Each individual tree in the random forest spits out a class prediction and the class with the most votes becomes our model’s prediction (see figure below).
<br>
<img src = "https://miro.medium.com/max/526/1*VHDtVaDPNepRglIAv72BFg.jpeg"><br>
The fundamental concept behind random forest is a simple but powerful one — the wisdom of crowds. In data science speak, the reason that the random forest model works so well is:
<b> A large number of relatively uncorrelated models (trees) operating as a committee will outperform any of the individual constituent models. </b>

The low correlation between models is the key. Just like how investments with low correlations (like stocks and bonds) come together to form a portfolio that is greater than the sum of its parts, uncorrelated models can produce ensemble predictions that are more accurate than any of the individual predictions. The reason for this wonderful effect is that the trees protect each other from their individual errors (as long as they don’t constantly all err in the same direction). While some trees may be wrong, many other trees will be right, so as a group the trees are able to move in the correct direction. So the prerequisites for random forest to perform well are:
<br><br>
1- There needs to be some actual signal in our features so that models built using those features do better than random guessing.
<br><br>
2- The predictions (and therefore the errors) made by the individual trees need to have low correlations with each other.

## <font color = "green"> <b>Question 4: </b></font> Conclusion

DT accuracy (avg)      : 0.7529 <br>
Bagging accuracy (avg) : 0.8100 <br>
Random forest (avg)    : 0.8549 <br><br>

<font color = "blue"><b>A large number of relatively uncorrelated models (trees) operating as a committee will outperform any of the individual constituent models.</b></font>
